Use Factorization Machines.

In [1]:
from os.path import expanduser
jupyter_td_file_path = expanduser("~") + "/.jupyter/conf"

import yaml
with open(jupyter_td_file_path, 'r') as f:
    doc = yaml.load(f)

API_KEY = doc['td']['apiKey']
DATABASE = doc['td']['db']

In [2]:
SETTINGS = {
  "RUN_TEST"    : True, # if run test or not
  "TEST_RATIO"  : 0.15, # how much data do you wanna use for testing
  "TRAIN_TABLE" : "kaggle.criteo2014_train",
  "TEST_TABLE"  : "kaggle.criteo2014_test",
  "RESOLUTION"  : 50, # how many categories each quantative features have
  "FACTOR"      : 20,
  "ITERATIONS"  : 5,
  "ETA"         : 0.01,
}

In [3]:
%matplotlib inline
from sklearn.metrics import roc_curve, auc, log_loss
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import pandas_td as td
import tdclient
import pandas as pd


con = td.connect(apikey=API_KEY, endpoint='https://api.treasuredata.com/')
engine_presto = con.query_engine(database=DATABASE, type='presto')
engine_hive = con.query_engine(database=DATABASE, type='hive')

td_client = tdclient.Client(API_KEY)

In [4]:
td.read_td("DROP TABLE IF EXISTS kaggle_criteo_fm_features", engine_presto)
td_client.create_log_table(DATABASE, "kaggle_criteo_fm_features")

td.read_td("DROP TABLE IF EXISTS kaggle_criteo_fm_model", engine_presto)
td_client.create_log_table(DATABASE, "kaggle_criteo_fm_model")

td.read_td("DROP TABLE IF EXISTS kaggle_criteo_fm_pred", engine_presto)
td_client.create_log_table(DATABASE, "kaggle_criteo_fm_pred")

True

###  Get max and min values for each quantative variable.
量的変数の最大値・最小値を取得

In [5]:
sql = '''
select
  min(min1) as min1, max(max1) as max1,
  min(min2) as min2, max(max2) as max2,
  min(min3) as min3, max(max3) as max3,
  min(min4) as min4, max(max4) as max4,
  min(min5) as min5, max(max5) as max5,
  min(min6) as min6, max(max6) as max6,
  min(min7) as min7, max(max7) as max7,
  min(min8) as min8, max(max8) as max8,
  min(min9) as min9, max(max9) as max9,
  min(min10) as min10, max(max10) as max10,
  min(min11) as min11, max(max11) as max11,
  min(min12) as min12, max(max12) as max12,
  min(min13) as min13, max(max13) as max13
from (
  select
    min(i1) as min1, max(i1) as max1, avg(i1) as avg1,
    min(i2) as min2, max(i2) as max2, avg(i2) as avg2,
    min(i3) as min3, max(i3) as max3, avg(i3) as avg3,
    min(i4) as min4, max(i4) as max4, avg(i4) as avg4,
    min(i5) as min5, max(i5) as max5, avg(i5) as avg5,
    min(i6) as min6, max(i6) as max6, avg(i6) as avg6,
    min(i7) as min7, max(i7) as max7, avg(i7) as avg7,
    min(i8) as min8, max(i8) as max8, avg(i8) as avg8,
    min(i9) as min9, max(i9) as max9, avg(i9) as avg9,
    min(i10) as min10, max(i10) as max10, avg(i10) as avg10,
    min(i11) as min11, max(i11) as max11, avg(i11) as avg11,
    min(i12) as min12, max(i12) as max12, avg(i12) as avg12,
    min(i13) as min13, max(i13) as max13, avg(i13) as avg13
  from
    {TRAIN_TABLE}
  union all
  select
    min(i1) as min1, max(i1) as max1, null as avg1,
    min(i2) as min2, max(i2) as max2, null as avg2,
    min(i3) as min3, max(i3) as max3, null as avg3,
    min(i4) as min4, max(i4) as max4, null as avg4,
    min(i5) as min5, max(i5) as max5, null as avg5,
    min(i6) as min6, max(i6) as max6, null as avg6,
    min(i7) as min7, max(i7) as max7, null as avg7,
    min(i8) as min8, max(i8) as max8, null as avg8,
    min(i9) as min9, max(i9) as max9, avg(i9) as avg9,
    min(i10) as min10, max(i10) as max10, avg(i10) as avg10,
    min(i11) as min11, max(i11) as max11, avg(i11) as avg11,
    min(i12) as min12, max(i12) as max12, avg(i12) as avg12,
    min(i13) as min13, max(i13) as max13, avg(i13) as avg13
  from
    {TEST_TABLE}
) t
'''.format_map(SETTINGS)

df_min_max = td.read_td(sql, engine_presto)

In [6]:
df_min_max

,min1,max1,min2,max2,min3,max3,min4,max4,min5,max5,...,min9,max9,min10,max10,min11,max11,min12,max12,min13,max13
0,0,5775,-3,257675,0,65535,0,969,0,23159456,...,0,29019,0,11,0,231,0,4008,0,7393


Treat all variables as a categorical feature.

In [7]:
sql = '''
INSERT OVERWRITE TABLE kaggle_criteo_fm_features

SELECT
  label,
  concat_array( -- don't need to add bias features for training data
    categorical_features(
      array('i1','i2','i3','i4','i5','i6','i7','i8','i9','i10','i11','i12','i13'),
      IF(i1 IS NULL, 'null', CAST(rescale(i1, {min1}, {max1}) * {RESOLUTION} AS INT)),
      IF(i2 IS NULL, 'null', CAST(rescale(i2, {min2}, {max2}) * {RESOLUTION} AS INT)),
      IF(i3 IS NULL, 'null', CAST(rescale(i3, {min3}, {max3}) * {RESOLUTION} AS INT)),
      IF(i4 IS NULL, 'null', CAST(rescale(i4, {min4}, {max4}) * {RESOLUTION} AS INT)),
      IF(i5 IS NULL, 'null', CAST(rescale(i5, {min5}, {max5}) * {RESOLUTION} AS INT)),
      IF(i6 IS NULL, 'null', CAST(rescale(i6, {min6}, {max6}) * {RESOLUTION} AS INT)),
      IF(i7 IS NULL, 'null', CAST(rescale(i7, {min7}, {max7}) * {RESOLUTION} AS INT)),
      IF(i8 IS NULL, 'null', CAST(rescale(i8, {min8}, {max8}) * {RESOLUTION} AS INT)),
      IF(i9 IS NULL, 'null', CAST(rescale(i9, {min9}, {max9}) * {RESOLUTION} AS INT)),
      IF(i10 IS NULL, 'null', CAST(rescale(i10, {min10}, {max10}) * {RESOLUTION} AS INT)),
      IF(i11 IS NULL, 'null', CAST(rescale(i11, {min11}, {max11}) * {RESOLUTION} AS INT)),
      IF(i12 IS NULL, 'null', CAST(rescale(i12, {min12}, {max12}) * {RESOLUTION} AS INT)),
      IF(i13 IS NULL, 'null', CAST(rescale(i13, {min13}, {max13}) * {RESOLUTION} AS INT))
    ),
    categorical_features(
      array('c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','c11','c12','c13','c14','c15','c16','c17','c18','c19','c20','c21','c22','c23','c24','c25','c26'),
      IF(c1 IS NULL, 'null', c1),
      IF(c2 IS NULL, 'null', c2),
      IF(c3 IS NULL, 'null', c3),
      IF(c4 IS NULL, 'null', c4),
      IF(c5 IS NULL, 'null', c5),
      IF(c6 IS NULL, 'null', c6),
      IF(c7 IS NULL, 'null', c7),
      IF(c8 IS NULL, 'null', c8),
      IF(c9 IS NULL, 'null', c9),
      IF(c10 IS NULL, 'null', c10),
      IF(c11 IS NULL, 'null', c11),
      IF(c12 IS NULL, 'null', c12),
      IF(c13 IS NULL, 'null', c13),
      IF(c14 IS NULL, 'null', c14),
      IF(c15 IS NULL, 'null', c15),
      IF(c16 IS NULL, 'null', c16),
      IF(c17 IS NULL, 'null', c17),
      IF(c18 IS NULL, 'null', c18),
      IF(c19 IS NULL, 'null', c19),
      IF(c20 IS NULL, 'null', c20),
      IF(c21 IS NULL, 'null', c21),
      IF(c22 IS NULL, 'null', c22),
      IF(c23 IS NULL, 'null', c23),
      IF(c24 IS NULL, 'null', c24),
      IF(c25 IS NULL, 'null', c25),
      IF(c26 IS NULL, 'null', c26)
    )
  ) as features
FROM 
  {TRAIN_TABLE}
'''.format_map({**SETTINGS, **(df_min_max.T[0].to_dict())})

td.read_td(sql, engine_hive)

,label,features


### Training with FM

In [8]:
sql = '''
INSERT OVERWRITE TABLE kaggle_criteo_fm_model

SELECT
  feature,
  AVG(Wi) as Wi,
  array_avg(Vif) as Vif
FROM (
  SELECT 
    train_fm(features, label, "-factor {FACTOR} -iters {ITERATIONS} -eta {ETA} -min 0 -max 1 -c") 
        AS (feature, Wi, Vif)
  FROM 
    kaggle_criteo_fm_features
) t
GROUP BY feature
'''.format_map({**SETTINGS, **(df_min_max.T[0].to_dict())})

td.read_td(sql, engine_hive)

,feature,wi,vif


### Prediction

In [9]:
sql = '''
WITH tmp_test AS (
SELECT
  id,
  add_bias(concat_array(
    categorical_features(
      array('i1','i2','i3','i4','i5','i6','i7','i8','i9','i10','i11','i12','i13'),
      IF(i1 IS NULL, 'null', CAST(rescale(i1, {min1}, {max1}) * {RESOLUTION} AS INT)),
      IF(i2 IS NULL, 'null', CAST(rescale(i2, {min2}, {max2}) * {RESOLUTION} AS INT)),
      IF(i3 IS NULL, 'null', CAST(rescale(i3, {min3}, {max3}) * {RESOLUTION} AS INT)),
      IF(i4 IS NULL, 'null', CAST(rescale(i4, {min4}, {max4}) * {RESOLUTION} AS INT)),
      IF(i5 IS NULL, 'null', CAST(rescale(i5, {min5}, {max5}) * {RESOLUTION} AS INT)),
      IF(i6 IS NULL, 'null', CAST(rescale(i6, {min6}, {max6}) * {RESOLUTION} AS INT)),
      IF(i7 IS NULL, 'null', CAST(rescale(i7, {min7}, {max7}) * {RESOLUTION} AS INT)),
      IF(i8 IS NULL, 'null', CAST(rescale(i8, {min8}, {max8}) * {RESOLUTION} AS INT)),
      IF(i9 IS NULL, 'null', CAST(rescale(i9, {min9}, {max9}) * {RESOLUTION} AS INT)),
      IF(i10 IS NULL, 'null', CAST(rescale(i10, {min10}, {max10}) * {RESOLUTION} AS INT)),
      IF(i11 IS NULL, 'null', CAST(rescale(i11, {min11}, {max11}) * {RESOLUTION} AS INT)),
      IF(i12 IS NULL, 'null', CAST(rescale(i12, {min12}, {max12}) * {RESOLUTION} AS INT)),
      IF(i13 IS NULL, 'null', CAST(rescale(i13, {min13}, {max13}) * {RESOLUTION} AS INT))
    ),
    categorical_features(
      array('c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','c11','c12','c13','c14','c15','c16','c17','c18','c19','c20','c21','c22','c23','c24','c25','c26'),
      IF(c1 IS NULL, 'null', c1),
      IF(c2 IS NULL, 'null', c2),
      IF(c3 IS NULL, 'null', c3),
      IF(c4 IS NULL, 'null', c4),
      IF(c5 IS NULL, 'null', c5),
      IF(c6 IS NULL, 'null', c6),
      IF(c7 IS NULL, 'null', c7),
      IF(c8 IS NULL, 'null', c8),
      IF(c9 IS NULL, 'null', c9),
      IF(c10 IS NULL, 'null', c10),
      IF(c11 IS NULL, 'null', c11),
      IF(c12 IS NULL, 'null', c12),
      IF(c13 IS NULL, 'null', c13),
      IF(c14 IS NULL, 'null', c14),
      IF(c15 IS NULL, 'null', c15),
      IF(c16 IS NULL, 'null', c16),
      IF(c17 IS NULL, 'null', c17),
      IF(c18 IS NULL, 'null', c18),
      IF(c19 IS NULL, 'null', c19),
      IF(c20 IS NULL, 'null', c20),
      IF(c21 IS NULL, 'null', c21),
      IF(c22 IS NULL, 'null', c22),
      IF(c23 IS NULL, 'null', c23),
      IF(c24 IS NULL, 'null', c24),
      IF(c25 IS NULL, 'null', c25),
      IF(c26 IS NULL, 'null', c26)
    )
  )) as features
FROM 
  {TEST_TABLE}
),

tmp_test_exploded AS (
SELECT 
  id,
  extract_feature(fv) AS feature,
  extract_weight(fv) AS Xi
FROM
  tmp_test t1 LATERAL VIEW explode(features) t2 AS fv
)

INSERT OVERWRITE TABLE kaggle_criteo_fm_pred

SELECT
  t.id AS Id,
  sigmoid(fm_predict(m.Wi, m.Vif, t.Xi)) AS Predicted
FROM 
  tmp_test_exploded t
  LEFT OUTER JOIN kaggle_criteo_fm_model m ON (t.feature = m.feature)
GROUP BY
  t.id
'''.format_map({**SETTINGS, **(df_min_max.T[0].to_dict())})

td.read_td(sql, engine_hive)

,id,predicted


## Result

Got a way worse score than the one with the baseline LR model.

- Baseline: 0.48396
- This notebook: 0.59276